<a href="https://colab.research.google.com/github/dinesh110598/Spin_glass_NN/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of different types of spin glass phases

Spin glass phases typically occur in frustrated spin systems where minimum energy cannot be attained in all lattice sites. They can occur in two possible situations:
1. Disorder in values of coupling constants. Eg: Edwards Anderson (EA) spin glass phase
2. Geometric constraints on neighbouring spin values. Eg: Antiferromagnetic triangular Ising glass phase

Main agenda: 
As it turns out, distinguishing these two kinds of spin glass phases is non-trivial and not possible by calculating quantities like magnetization, susceptibility, two point correlations. Our aim will be to feed simulated thermal ensembles of EA spin glass and triangular antiferromagnetic spin glass labelled categorically, to a convolutional neural network and see whether it accurately learns the inherent difference in their statistics 

# Classify Square Uniform EA and Triangular AFM lattices

In [ ]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   2449      0 --:--:-- --:--:-- --:--:--  2445
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 348.0MB 24kB/s 
+ set +ex
Installation succeeded!


In [ ]:
import math
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import tensorflow.keras as tfk
import tensorflow as tf
from numba import cuda, float32, int32

In [ ]:
from google.colab import drive
from google.colab import output
drive.mount('/content/drive', force_remount=True)
folder_path = "/content/drive/My Drive/Project Presentations/Spin_glass_phase_classification/"

Mounted at /content/drive


### Necessary cuda kernels

In [ ]:
@cuda.jit
def update_red(spin, seed, T):
    J = -1.0  # Nearest neighbour coupling constant

    def random_uniform(x, y, z):
        seed[x, y, z] = np.int32((seed[x, y, z]*1664525 + 1013904223) % 2**31)
        return seed[x, y, z] / (2**31)

    def bvc (x):
        if x == spin.shape[0]:
            x = 0
        return x       

    def sum_nn(x, y, z):  # This adds spins of six neighbours instead of 4 subject to
        #many constraints characteristic of triangular lattices

        value = 0
        value += spin[bvc(x+1), y, z]
        if (x % 2 == 0):
            value += spin[bvc(x+1), y-1, z]
        else:
            value += spin[bvc(x+1), bvc(y+1), z]

        value += spin[x-1, y, z]
        if (x % 2 == 0):
            value += spin[x-1, y-1, z]
        else:
            value += spin[x-1, bvc(y+1), z]

        value += spin[x, bvc(y+1), z]
        value += spin[x, y-1, z]

        return value

    def calc(x, y, z):
        probs = random_uniform(x, y, z)
        if (probs < math.exp(-2*J*spin[x, y, z]*sum_nn(x, y, z)/T[0])):
            spin[x, y, z] *= np.int8(-1)

    x, y, z = cuda.grid(3)
    p, q = x % 3, y % 2

    if x < spin.shape[0] and y < spin.shape[1] and z < spin.shape[2]:
        if (p == 0 and q == 0) or (p == 1 and q == 1):
            calc(x, y, z)

@cuda.jit
def update_blue(spin, seed, T):
    J = -1.0  # Nearest neighbour coupling constant

    def random_uniform(x, y, z):
        seed[x, y, z] = np.int32((seed[x, y, z]*1664525 + 1013904223) % 2**31)
        return seed[x, y, z] / (2**31)

    def bvc (x):
        if x == spin.shape[0]:
            x = 0
        return x

    def sum_nn(x, y, z):  # This adds spins of six neighbours instead of 4 subject to
        #many constraints characteristic of triangular lattices

        value = 0
        value += spin[bvc(x+1), y, z]
        if (x % 2 == 0):
            value += spin[bvc(x+1), y-1, z]
        else:
            value += spin[bvc(x+1), bvc(y+1), z]

        value += spin[x-1, y, z]
        if (x % 2 == 0):
            value += spin[x-1, y-1, z]
        else:
            value += spin[x-1, bvc(y+1), z]

        value += spin[x, bvc(y+1), z]
        value += spin[x, y-1, z]

        return value

    def calc(x, y, z):
        probs = random_uniform(x, y, z)
        if (probs < math.exp(-2*J*spin[x, y, z]*sum_nn(x, y, z)/T[0])):
            spin[x, y, z] *= np.int8(-1)

    x, y, z = cuda.grid(3)
    p, q = x % 3, y % 2

    if x < spin.shape[0] and y < spin.shape[1] and z < spin.shape[2]:
        if (p == 1 and q == 0) or (p == 2 and q == 1):
            calc(x, y, z)

@cuda.jit
def update_green (spin, seed, T):
    J = -1.0  # Nearest neighbour coupling constant

    def random_uniform(x, y, z):
        seed[x, y, z] = np.int32((seed[x, y, z]*1664525 + 1013904223) % 2**31)
        return seed[x, y, z] / (2**31)

    def bvc (x):
        if x == spin.shape[0]:
            x = 0
        return x

    def sum_nn(x, y, z):  # This adds spins of six neighbours instead of 4 subject to
        #many constraints characteristic of triangular lattices

        value = 0
        value += spin[bvc(x+1), y, z]
        if (x % 2 == 0):
            value += spin[bvc(x+1), y-1, z]
        else:
            value += spin[bvc(x+1), bvc(y+1), z]

        value += spin[x-1, y, z]
        if (x % 2 == 0):
            value += spin[x-1, y-1, z]
        else:
            value += spin[x-1, bvc(y+1), z]

        value += spin[x, bvc(y+1), z]
        value += spin[x, y-1, z]

        return value

    def calc(x, y, z):
        probs = random_uniform(x, y, z)
        if (probs < math.exp(-2*J*spin[x, y, z]*sum_nn(x, y, z)/T[0])):
            spin[x, y, z] *= np.int8(-1)

    x, y, z = cuda.grid(3)
    p, q = x % 3, y % 2

    if x < spin.shape[0] and y < spin.shape[1] and z < spin.shape[2]:
        if (p == 2 and q == 0) or (p == 0 and q == 1):
            calc(x, y, z)

@cuda.jit
def calc_energy_Tri (spin, energy):
    def bvc (x):
        if x == spin.shape[1]:
            x = 0
        return x
    
    def sum_nn(x, y, z):  # This adds spins of six neighbours instead of 4 subject to
        #many constraints characteristic of triangular lattices

        value = 0
        value += spin[bvc(x+1), y, z]
        if (x % 2 == 0):
            value += spin[bvc(x+1), y-1, z]
        else:
            value += spin[bvc(x+1), bvc(y+1), z]

        value += spin[x-1, y, z]
        if (x % 2 == 0):
            value += spin[x-1, y-1, z]
        else:
            value += spin[x-1, bvc(y+1), z]

        value += spin[x, bvc(y+1), z]
        value += spin[x, y-1, z]

        return value
    
    ener = 0
    z = cuda.grid (1)
    for x in range (spin.shape[0]):
        for y in range (spin.shape[1]):
            ener -= spin[x,y,z]*sum_nn(x,y,z)
    energy[z] = 0.5*ener

In [ ]:
@cuda.jit
def update_black(spin, seed, T, Jnn):
    def random_uniform(x, y, z):
        seed[x, y, z] = np.int32((seed[x, y, z]*1664525 + 1013904223) % 2**31)
        return seed[x, y, z] / (2**31)

    def bvc(x):
        if x == spin.shape[0]:
            return 0
        else:
            return x

    def sum_nn(x, y, z):
        sum = 0
        sum += Jnn[x, y, 0]*spin[bvc(x+1), y, z]
        sum += Jnn[x, y, 1]*spin[x, bvc(y+1), z]
        sum += Jnn[x-1, y, 0]*spin[x-1, y, z]
        sum += Jnn[x, y-1, 1]*spin[x, y-1, z]
        return sum

    def calc(x, y, z):
        probs = random_uniform(x, y, z)
        delta = 2*spin[x, y, z]*sum_nn(x, y, z)
        if probs < math.exp(-delta/T[0]):
            spin[x, y, z] *= np.int8(-1)
            #energy[z] += delta

    x, y, z = cuda.grid(3)
    p, q = x % 2, y % 2
    if x < spin.shape[0] and y < spin.shape[1] and z < spin.shape[2]:
        if (p == 0 and q == 0) or (p == 1 and q == 1):
            calc(x, y, z)

@cuda.jit
def update_white(spin, seed, T, Jnn):
    def random_uniform(x, y, z):
        seed[x, y, z] = np.int32((seed[x, y, z]*1664525 + 1013904223) % 2**31)
        return seed[x, y, z] / (2**31)

    def bvc(x):
        if x == spin.shape[0]:
            return 0
        else:
            return x

    def sum_nn(x, y, z):
        sum = 0
        sum += Jnn[x, y, 0]*spin[bvc(x+1), y, z]
        sum += Jnn[x, y, 1]*spin[x, bvc(y+1), z]
        sum += Jnn[x-1, y, 0]*spin[x-1, y, z]
        sum += Jnn[x, y-1, 1]*spin[x, y-1, z]
        return sum

    def calc(x, y, z):
        probs = random_uniform(x, y, z)
        delta = 2*spin[x, y, z]*sum_nn(x, y, z)
        if probs < math.exp(-delta/T[0]):
            spin[x, y, z] *= np.int8(-1)
            #energy[z] += delta

    x, y, z = cuda.grid(3)
    p, q = x % 2, y % 2
    if x < spin.shape[0] and y < spin.shape[1] and z < spin.shape[2]:
        if (p == 1 and q == 0) or (p == 0 and q == 1):
            calc(x, y, z)

@cuda.jit
def parallel_temper(spin, T, seed, energy):
    z = cuda.grid(1)

    rand_n = 0 if float32(seed[0, 0, 0]/2**31) < 0.5 else 1
    ptr = 2*z + rand_n
    if ptr < energy.shape[0]-1:
        val0 = 1./T[ptr]
        val1 = 1./T[ptr+1]
        e0 = energy[ptr]
        e1 = energy[ptr+1]
        rand_unif = float32(seed[0, 1, z] / 2**31)
        arg = (e0 - e1)*(val0 - val1)
        if (arg < 0):
            if rand_unif < math.exp(arg):
                T[ptr] = 1/val1
                T[ptr+1] = 1/val0
        else:
            T[ptr] = 1/val1
            T[ptr+1] = 1/val0

@cuda.jit
def calc_energy (spin, J_nn, energy): #m_ threads on a single block...
    def bvc(x):
        if x == spin.shape[1]:
            return 0
        else:
            return x

    z = cuda.grid (1)
    len_ = spin.shape[1]
    ener = 0
    for x in range(len_):
        for y in range(len_):
            ener -= J_nn[x,y,0] * spin[x,y,z]*spin[bvc(x+1),y,z]
            ener -= J_nn[x,y,1] * spin[x,y,z]*spin[x,bvc(y+1),z]

    energy[z] = ener

### Simulator class

In [ ]:
class SpinGlassData ():
    def __init__(self, lat_len= 48, ens_size=80):
        self.shape = (lat_len, lat_len)
        self.m = ens_size
        self.tpb = (8,8,1)
        self.spin = cp.random.choice ([1,-1], self.shape+(self.m,)).astype(np.int8)
        self.spin2 = cp.random.choice ([1,-1], self.shape+(self.m//2,)).astype(np.int8)
        self.seed = cp.random.randint (-10000,10000, size=self.shape+(self.m,),
                                       dtype=np.int32)
        self.energy = cp.zeros (self.m, np.float32)
        self.energy2 = cp.zeros (self.m//2, np.float32)
    
    def equilib_EA (self):
        temp = 0.5
        J_nn = cp.random.uniform (-1.73205, 1.73205, self.shape+(2,), dtype=np.float32)
        vr_T = cp.linspace (temp, temp+2.5, self.m, dtype=np.float32)
        bpg = (self.shape[0]//self.tpb[0],self.shape[1]//self.tpb[1], self.m)
        for _ in range(250):
            for _ in range (5):
                update_black[bpg,self.tpb] (self.spin, self.seed, vr_T, J_nn)
                update_white[bpg,self.tpb] (self.spin, self.seed, vr_T, J_nn)
            calc_energy[1, self.m](self.spin, J_nn, self.energy)
            parallel_temper[1, self.m//2] (self.spin, vr_T, self.seed, self.energy)
        vr_T = cp.full (self.m, temp, np.float32)
        for _ in range (500):
            update_black[bpg, self.tpb] (self.spin, self.seed, vr_T, J_nn)
            update_white[bpg, self.tpb] (self.spin, self.seed, vr_T, J_nn)
        calc_energy[1, self.m](self.spin, J_nn, self.energy)
        order = cp.asnumpy(cp.argsort (self.energy))
        return cp.asnumpy(self.spin)[...,order[:self.m//2]]

    def equilib_TriAFM (self):
        temp = 0.5
        bpg = (self.shape[0]//self.tpb[0], self.shape[1]//self.tpb[1], self.m)
        vr_T = cp.array ([temp])
        for _ in range (500):
            update_red[bpg, self.tpb] (self.spin2, self.seed, vr_T)
            update_blue[bpg, self.tpb] (self.spin2, self.seed, vr_T)
            update_green[bpg, self.tpb] (self.spin2, self.seed, vr_T)
        calc_energy_Tri[1, self.m//2](self.spin2, self.energy2)
        order = cp.asnumpy(cp.argsort (self.energy2))
        return cp.asnumpy(self.spin2)[...,order]
    
    def generate_train_data (self, train_len=2000):
        t_lattice = []
        t_label = []
        for _ in range (train_len//2):
            t_lattice.append (self.equilib_EA())
            t_label.append (0)
        for _ in range (train_len//2):
            t_lattice.append (self.equilib_TriAFM())
            t_label.append (1)
        t_lattice = (0.5*np.stack (t_lattice)).astype (np.float32)
        t_label = np.stack (t_label).astype (np.int8)
        t_data = tf.data.Dataset.from_tensor_slices ((t_lattice, t_label))
        return t_data.shuffle (buffer_size=train_len)

### Data generation

In [ ]:
simulator = SpinGlassData ()

In [ ]:
train_data = simulator.generate_train_data (500)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

### Saving and Loading Data

In [ ]:
def save_dataset (num, train_data):
    def serialize_example(lattice, label):
        """
        Creates a tf.train.Example message ready to be written to a file.
        """

        def _bytes_feature(value):
            """Returns a bytes_list from a string / byte."""
            #value = tf.io.serialize_tensor(value)
            if isinstance(value, type(tf.constant(0))):
                value = tf.io.serialize_tensor(value).numpy()
            return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

        def _int64_feature(value):
            """Returns an int64_list from a bool / enum / int / uint."""
            return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

        # Create a dictionary mapping the feature name to the tf.train.Example-compatible
        # data type.
        feat = { 
            'lattice': _bytes_feature(lattice),
            'label': _int64_feature(label),
        }
        # Create a Features message using tf.train.Example.
        example_proto = tf.train.Example(features=tf.train.Features(feature=feat))
        return tf.reshape(example_proto.SerializeToString(),())

    def tf_serialize_example(lattice,label):
        tf_string = tf.py_function(
            serialize_example,
            (lattice,label),  # pass these args to the above function.
            tf.string)      # the return type is `tf.string`.
        return tf.reshape(tf_string, ()) # The result is a scalar


    serialised_dataset = train_data.map (tf_serialize_example)

    filename = folder_path+'Training Data/EA_TriAFM2d_48_500('+num+').tfrecord'
    writer = tf.data.experimental.TFRecordWriter(filename)
    writer.write(serialised_dataset)

save_dataset ('4',train_data)

In [ ]:
def extract_dataset (num):
    filename = folder_path+'Training Data/EA_TriAFM2d_48_500('+num+').tfrecord'
    raw_dataset = tf.data.TFRecordDataset(filename)

    def _parse_function(example_proto):
        # Parse the input `tf.train.Example` proto using the dictionary below.
        feature_description = { 
            'lattice': tf.io.FixedLenFeature([], tf.string, default_value=''),
            'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        }
        return tf.io.parse_single_example(example_proto, feature_description)

    parsed_dataset = raw_dataset.map(_parse_function)

    def lat_parser (record):
        record = (tf.ensure_shape (tf.io.parse_tensor(record['lattice'],tf.float32),
                                   simulator.shape+(simulator.m//2,)),
                  record['label'])
        return record
    dataset = parsed_dataset.map (lat_parser)
    return dataset

train_data = extract_dataset ('1')
train_data = train_data.concatenate(extract_dataset ('2'))
train_data = train_data.concatenate(extract_dataset ('3'))
train_data = train_data.concatenate(extract_dataset ('4'))

In [ ]:
val_data = train_data.take (400)
train_data = train_data.skip (400)
train_data = train_data.batch (8)
val_data = val_data.batch (8)

## Neural network initialization and training

In [ ]:
brain = tfk. Sequential([
    tfk.layers.Conv2D(64, (2,2), activation='relu', input_shape = simulator.shape+(simulator.m//2,)),
    tfk.layers.MaxPool2D (),
    tfk.layers.Conv2D(64, (2,2), activation='relu'),
    tfk.layers.Flatten(),
    tfk.layers.Dense(64, activation='relu'),
    tfk.layers.Dropout(0.3),
    tfk.layers.Dense(2, activation='softmax')
])

In [ ]:
brain.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist = brain.fit (train_data, epochs=1, validation_data=val_data)

200/200 [==============================] - 3s 14ms/step - loss: 0.5319 - accuracy: 0.7375 - val_loss: 0.0849 - val_accuracy: 1.0000


In [ ]:
brain.save(folder_path+'NN Models/EA_TriAFM2d_48.h5')

Testing neural network accuracy

In [ ]:
test_data = simulator.generate_train_data (200)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
test_data = test_data.batch (8)
brain.evaluate (test_data)

25/25 [==============================] - 0s 4ms/step - loss: 0.0914 - accuracy: 1.0000


[0.09139034152030945, 1.0]

We have managed to train a neural network that can classify cold phases of Square Edwards Anderson and Triangular antiferromagnetic models with 100% accuracy...